In [1]:
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
model_dir = 'code_submission/'                        # Change the model to a better one once you have one!
result_dir = 'results_submission/' 
problem_dir = 'ingestion_program/'  
score_dir = 'scoring_program/'
from sys import path; path.append(model_dir); path.append(problem_dir); path.append(score_dir); 
%matplotlib inline
# Uncomment the next lines to auto-reload libraries (this causes some problem with pickles in Python 3)
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from data_io import read_as_df
import numpy as np
from matplotlib import pyplot as plt
from data_manager import DataManager

In [3]:
batch_size=20
def extract_features(generator, sample_count, include_labels=True):
    batch_size = 20
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = generator
    i=0
    if include_labels:
        for inputs_batch, labels_batch in generator:
            features_batch = conv_base.predict(inputs_batch)
            features[i * batch_size : (i+1) * batch_size] = features_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
            i += 1
            if i*batch_size >= sample_count:
                break
        return features, labels
    else:
        for inputs_batch in generator:
            features_batch = conv_base.predict(inputs_batch)
            features[i * batch_size : (i+1) * batch_size] = features_batch
            i += 1
            if i*batch_size >= sample_count:
                break
        return features

def plot_loss_acc(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1,len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [4]:
data_dir = '../input_data'
data_name = 'microscopy'
!ls $data_dir*

microscopyCount_feat.name      microscopy_feat.name
microscopyCount_label.name     microscopy_label.name
microscopyCount_test.data      microscopy_test.data
microscopyCount_train.data     microscopy_train.data
microscopyCount_train.solution microscopy_train.solution
microscopyCount_valid.data     microscopy_valid.data


In [4]:
D = DataManager(data_name, data_dir, replace_missing=True)
print(D)

Info file NOT found : /Users/Lawrence/Documents/MACHINE_LEARNING/DSA/input_data/microscopy_public.info
DataManager : microscopy
info:
	usage = No Info File
	name = microscopy
	format = dense
	is_sparse = 0
	has_categorical = 0
	has_missing = 0
	target_num = 1
	label_num = 2
	task = binary.classification
	target_type = Binary
	metric = auc_metric
	feat_type = Mixed
	feat_num = 4800
	train_num = 23809
	valid_num = 11893
	test_num = 12101
	time_budget = 600
data:
	X_train = array(23809, 4800)
	Y_train = array(23809,)
	X_valid = array(11893, 4800)
	Y_valid = array(0,)
	X_test = array(12101, 4800)
	Y_test = array(0,)
feat_type:	array(4800,)
feat_idx:	array(0,)



In [44]:
X_test = D.data['X_test']
X_validation = D.data['X_valid']

NameError: name 'D' is not defined

In [5]:
import pandas as pd

In [6]:
train_data = pd.read_csv("./train_data.csv")

In [22]:
testset = pd.DataFrame(X_test)
validset = pd.DataFrame(X_validation)
testset.to_csv("test_data.csv",index=False, header=True)
validset.to_csv("validation_data.csv",index=False, header=True)

In [46]:
#train_data = trainset
train_x = train_data.drop("target", axis=1).iloc[:6000]
train_y = train_data.target.iloc[:6000]
validation_x = train_data.drop("target", axis=1).iloc[6000:10000]
validation_y = train_data.target.iloc[6000:10000]
test_x = train_data.drop("target", axis=1).iloc[10000:]
test_y = train_data.target.iloc[10000:]

In [47]:
train_x = train_x.values
train_y = train_y.values
validation_x = validation_x.values
validation_y = validation_y.values
test_x = test_x.values
test_y = test_y.values

In [53]:
y = Y_train
X = X_train.reshape((len(X_train), 40, 40, 3))
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
train_generator = train_datagen.flow(X, y=y, batch_size=20,shuffle=False)
x_val, y_val = validation_data
x_val = x_val.reshape((len(x_val), 40,40,3))
val_data = (x_val, y_val)
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow(x_val, y=y_val, batch_size=20, shuffle=False)

NameError: name 'Y_train' is not defined

In [58]:
X_validation = D.data['X_valid']
X_test = D.data['X_test']
validation_x  = X_validation.reshape((len(X_validation), 40,40,3))
test_x  = X_test.reshape((len(X_test), 40,40,3))
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow(validation_x,batch_size=20)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator =test_datagen.flow(test_x,batch_size=20)

In [10]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

In [60]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(40,40,3))

In [ ]:
train_features, train_labels = extract_features(train_generator, len(X_train))
validation_features, validation_labels = extract_features(valid_generator, len(x_val))
# flatten extracted features to be able to feed them to a densely connected layer
train_features = np.reshape(train_features, (len(X_train), 4*4*512))
validation_features = np.reshape(validation_features, (len(x_val), 4* 4*512))

# Train Model

In [48]:
from data_io import write
from model import model

In [52]:
M = model()

In [53]:
M.model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 38, 38, 32)        896       
_________________________________________________________________
batch_normalization_18 (Batc (None, 38, 38, 32)        128       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 19, 19, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 17, 17, 64)        18496     
_________________________________________________________________
batch_normalization_19 (Batc (None, 17, 17, 64)        256       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 8, 8, 64)         

In [54]:
M.fit(train_x, train_y, (validation_x,validation_y),epochs=10)

Epoch 1/10
186/187 [============================>.] - ETA: 0s - loss: 0.2135 - accuracy: 0.9182

ValueError: Empty training data.

In [18]:
M.fit(train_x, train_y,(validation_x,validation_y), epochs=10)

Epoch 1/10
300/300 [==============================] - 21s 71ms/step - loss: 0.2462 - accuracy: 0.9070 - val_loss: 37.1331 - val_accuracy: 0.4500
Epoch 2/10
300/300 [==============================] - 16s 53ms/step - loss: 0.1511 - accuracy: 0.9480 - val_loss: 2.3999 - val_accuracy: 0.9000
Epoch 3/10
300/300 [==============================] - 19s 63ms/step - loss: 0.1160 - accuracy: 0.9602 - val_loss: 0.6305 - val_accuracy: 1.0000
Epoch 4/10
300/300 [==============================] - 18s 61ms/step - loss: 0.1068 - accuracy: 0.9627 - val_loss: 9.5782 - val_accuracy: 0.6000
Epoch 5/10
300/300 [==============================] - 17s 58ms/step - loss: 0.1046 - accuracy: 0.9678 - val_loss: 1.6834 - val_accuracy: 0.9000
Epoch 6/10
300/300 [==============================] - 18s 59ms/step - loss: 0.0928 - accuracy: 0.9672 - val_loss: 0.1673 - val_accuracy: 1.0000
Epoch 7/10
300/300 [==============================] - 18s 60ms/step - loss: 0.0871 - accuracy: 0.9722 - val_loss: 0.1441 - val_accuracy

In [36]:
Y_hat_train = M.predict(train_x) # Optional, not really needed to test on taining examples
Y_hat_valid = M.predict(validation_x)
Y_hat_test = M.predict(test_x)

In [37]:
from sklearn.metrics import roc_auc_score
metric_name, scoring_function = "auc_binary", roc_auc_score
print('Using scoring metric:', metric_name)


Using scoring metric: auc_binary


In [38]:
print('Training score for the', metric_name, 'metric = %5.4f' % scoring_function(train_y, Y_hat_train))
print('Valid score for the', metric_name, 'metric = %5.4f' % scoring_function(validation_y, Y_hat_valid))
print('Test score for the', metric_name, 'metric = %5.4f' % scoring_function(test_y, Y_hat_test))
print('Ideal score for the', metric_name, 'metric = %5.4f' % scoring_function(train_y, train_y))

Training score for the auc_binary metric = 0.9904
Valid score for the auc_binary metric = 0.9943
Test score for the auc_binary metric = 0.9884
Ideal score for the auc_binary metric = 1.0000


In [80]:
f = []
with open("./results_submission/microscopy_test.predict") as file:
    f.append(file.readline())

In [28]:
from data_io import read_as_df

In [33]:
test_sol = pd.read_csv("../input_data/microscopy_train.solution", header=None)
test_sol

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [39]:
train_pred = pd.read_csv("./results_submission/microscopy_valid.predict", header=None)
test_sol = pd.read_csv("../input_data/microscopy_valid.solution", header=None)
train_pred["actual"] = test_sol
train_pred['residual'] = train_pred.actual - train_pred[0]
train_pred[train_pred.residual > 0.5]

FileNotFoundError: File b'../input_data/microscopy_valid.solution' does not exist

In [176]:
train_pred.columns = ["pred","orig"]

In [177]:
roc_auc_score(train_pred.pred, train_pred.orig)

ValueError: continuous format is not supported

In [199]:
A = np.random.randn(4,3)
B = np.sum(A, axis = 1, keepdims = True)

In [200]:
B.shape

(4, 1)

In [201]:
A

array([[ 4.59022552e-01,  1.67234203e+00, -5.61013905e-01],
       [ 6.94189495e-01,  1.74559241e-03,  1.20446870e+00],
       [-1.95964661e-01,  1.63534866e+00,  1.86728684e+00],
       [-1.26310632e+00, -1.50413554e-01, -1.80324292e-01]])

In [202]:
B

array([[ 1.57035068],
       [ 1.90040379],
       [ 3.30667084],
       [-1.59384417]])